# Кейс 1: Предсказание оттока клиентов кредитных карт

**Цель**: Построить модель бинарной классификации для предсказания оттока клиентов банка.

**Датасет**: [Credit Card Customers](https://www.kaggle.com/datasets/whenamancodes/credit-card-customers-prediction)

---

## 1. Импорты

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, recall_score, precision_score, f1_score, accuracy_score

import warnings
warnings.filterwarnings('ignore')

# Настройки отображения
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

## 2. Загрузка и первичный осмотр данных

In [ ]:
df = pd.read_csv('data/BankChurners.csv')

print('Размер датасета:', df.shape)
print(f'\nСтрок: {df.shape[0]}, Столбцов: {df.shape[1]}')

In [ ]:
# Первые строки
df.head()

In [ ]:
# Типы данных
print('Типы данных:')
print(df.dtypes)

In [ ]:
# Базовая статистика
df.describe()

In [ ]:
# Информация о датасете
df.info()

## 3. Exploratory Data Analysis (EDA)

### 3.1 Проверка пропусков

In [ ]:
missing = df.isnull().sum()
missing_pct = (df.isnull().sum() / len(df) * 100).round(2)

missing_df = pd.DataFrame({
    'Пропуски': missing,
    'Процент': missing_pct
})

print('Пропущенные значения:')
print(missing_df[missing_df['Пропуски'] > 0] if missing_df['Пропуски'].sum() > 0 else 'Пропусков нет!')

### 3.2 Удаление ненужных колонок

In [ ]:
# Смотрим названия колонок
print('Все колонки:')
for i, col in enumerate(df.columns):
    print(f'{i+1}. {col}')

In [ ]:
# Удаляем:
# - CLIENTNUM — ID клиента, не несёт информации для модели
# - Naive_Bayes_* — служебные колонки Kaggle

cols_to_drop = ['CLIENTNUM']

# Находим служебные колонки Naive_Bayes
naive_bayes_cols = [col for col in df.columns if 'Naive_Bayes' in col]
cols_to_drop.extend(naive_bayes_cols)

print('Удаляем колонки:')
for col in cols_to_drop:
    print(f'  - {col}')

df = df.drop(columns=cols_to_drop, errors='ignore')
print(f'\nОсталось колонок: {len(df.columns)}')

### 3.3 Анализ целевой переменной

In [ ]:
print('Распределение целевой переменной Attrition_Flag:')
print(df['Attrition_Flag'].value_counts())
print('\nВ процентах:')
print((df['Attrition_Flag'].value_counts(normalize=True) * 100).round(2))

In [ ]:
plt.figure(figsize=(8, 5))
ax = df['Attrition_Flag'].value_counts().plot(kind='bar', color=['#2ecc71', '#e74c3c'], edgecolor='black')
plt.title('Баланс классов (Attrition_Flag)', fontsize=14)
plt.xlabel('Статус клиента')
plt.ylabel('Количество')
plt.xticks(rotation=0)

# Добавляем подписи
for i, v in enumerate(df['Attrition_Flag'].value_counts()):
    ax.text(i, v + 100, str(v), ha='center', fontsize=12)

plt.tight_layout()
plt.show()

print('\n⚠️ Вывод: Дисбаланс классов ~16% vs 84%')
print('Решение: будем использовать class_weight="balanced" в моделях')

### 3.4 Анализ числовых признаков

In [ ]:
# Выделяем числовые признаки
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print(f'Числовых признаков: {len(numeric_cols)}')
print(numeric_cols)

In [ ]:
# Боксплоты для ключевых числовых признаков
key_numeric = ['Customer_Age', 'Credit_Limit', 'Total_Trans_Amt', 'Total_Trans_Ct', 
               'Total_Revolving_Bal', 'Avg_Utilization_Ratio']

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, col in enumerate(key_numeric):
    df.boxplot(column=col, ax=axes[i])
    axes[i].set_title(col, fontsize=12)

plt.suptitle('Боксплоты числовых признаков', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Распределения по классам
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, col in enumerate(key_numeric):
    for label in df['Attrition_Flag'].unique():
        subset = df[df['Attrition_Flag'] == label][col]
        axes[i].hist(subset, alpha=0.5, label=label, bins=30)
    axes[i].set_title(col)
    axes[i].legend()

plt.suptitle('Распределения признаков по классам', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

### 3.5 Анализ категориальных признаков

In [ ]:
# Выделяем категориальные признаки
cat_cols = df.select_dtypes(include=['object']).columns.tolist()
cat_cols.remove('Attrition_Flag')  # Убираем целевую

print(f'Категориальных признаков: {len(cat_cols)}')
print(cat_cols)

In [ ]:
# Уникальные значения категориальных признаков
for col in cat_cols:
    print(f'\n{col}:')
    print(df[col].value_counts())

In [ ]:
# Визуализация категориальных признаков
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, col in enumerate(cat_cols):
    df[col].value_counts().plot(kind='bar', ax=axes[i], edgecolor='black')
    axes[i].set_title(col)
    axes[i].tick_params(axis='x', rotation=45)

# Убираем лишний subplot если есть
if len(cat_cols) < 6:
    axes[-1].axis('off')

plt.suptitle('Распределения категориальных признаков', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

### 3.6 Корреляционная матрица

In [ ]:
# Корреляция числовых признаков
plt.figure(figsize=(14, 12))
corr = df.select_dtypes(include=[np.number]).corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
sns.heatmap(corr, mask=mask, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=0.5)
plt.title('Корреляционная матрица числовых признаков', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Сильные корреляции (> 0.5)
print('Сильные корреляции (|r| > 0.5):')
for i in range(len(corr.columns)):
    for j in range(i+1, len(corr.columns)):
        if abs(corr.iloc[i, j]) > 0.5:
            print(f'  {corr.columns[i]} <-> {corr.columns[j]}: {corr.iloc[i, j]:.2f}')

### 3.7 Итоговая таблица выводов по EDA

In [ ]:
eda_summary = pd.DataFrame({
    'Категория': ['ID', 'Служебные', 'Целевая', 'Категориальные', 'Числовые', 'Дисбаланс'],
    'Признак': ['CLIENTNUM', 'Naive_Bayes_*', 'Attrition_Flag', 'Gender, Education и др.', 'Customer_Age и др.', 'Attrition_Flag'],
    'Действие': ['Удалить', 'Удалить', 'Бинаризовать', 'LabelEncoder', 'StandardScaler', 'class_weight=balanced'],
    'Причина': ['Не несёт информации', 'Служебные колонки Kaggle', '0/1 для модели', 'Преобразование для ML', 'Для LogReg', '16% vs 84%']
})

print('=' * 80)
print('ИТОГИ EDA')
print('=' * 80)
print(eda_summary.to_string(index=False))

## 4. Preprocessing (Предобработка данных)

In [ ]:
# Копируем датафрейм для обработки
df_processed = df.copy()

# Кодируем целевую переменную
df_processed['Attrition_Flag'] = df_processed['Attrition_Flag'].map({
    'Existing Customer': 0,
    'Attrited Customer': 1
})

print('Целевая переменная закодирована:')
print(df_processed['Attrition_Flag'].value_counts())

In [ ]:
# Кодируем категориальные признаки (отдельный encoder для каждой колонки)
encoders = {}

print('Кодирование категориальных признаков:')
for col in cat_cols:
    encoders[col] = LabelEncoder()
    df_processed[col] = encoders[col].fit_transform(df_processed[col].astype(str))
    print(f'  {col}: {len(df[col].unique())} уникальных значений')

print('\nГотово!')

In [ ]:
# Проверяем результат
df_processed.head()

In [ ]:
# Разделяем на X и y
X = df_processed.drop('Attrition_Flag', axis=1)
y = df_processed['Attrition_Flag']

print(f'X shape: {X.shape}')
print(f'y shape: {y.shape}')
print(f'\nПризнаки: {list(X.columns)}')

In [ ]:
# Train/test split (stratified из-за дисбаланса классов)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    stratify=y, 
    random_state=42
)

print(f'Train set: {X_train.shape[0]} samples')
print(f'Test set: {X_test.shape[0]} samples')
print(f'\nБаланс в train: {y_train.value_counts(normalize=True).round(3).to_dict()}')
print(f'Баланс в test: {y_test.value_counts(normalize=True).round(3).to_dict()}')

In [ ]:
# Масштабируем признаки для Logistic Regression
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print('Масштабирование выполнено (StandardScaler)')

## 5. Выбор метрики

In [ ]:
metrics_explanation = pd.DataFrame({
    'Метрика': ['Accuracy', 'Precision', 'Recall', 'F1-score'],
    'Формула': ['(TP+TN)/(TP+TN+FP+FN)', 'TP/(TP+FP)', 'TP/(TP+FN)', '2*P*R/(P+R)'],
    'Когда использовать': [
        'Сбалансированные классы',
        'Важно не тратить ресурсы зря (False Positive дорого)',
        'Важно найти ВСЕХ положительных (False Negative дорого)',
        'Компромисс между Precision и Recall'
    ],
    'Наш случай': ['❌ Дисбаланс', '❌', '✅ ВЫБИРАЕМ', '❌']
})

print('=' * 80)
print('ВЫБОР МЕТРИКИ')
print('=' * 80)
print(metrics_explanation.to_string(index=False))

### Обоснование выбора Recall

**Бизнес-контекст**: Предсказание оттока клиентов банка.

**Почему Recall?**
- **False Negative (FN)** = клиент уйдёт, но модель этого не предскажет → **потеря клиента и прибыли**
- **False Positive (FP)** = клиент не уйдёт, но модель предскажет уход → **лишнее удержание**, но клиент останется

**Вывод**: Пропустить уходящего клиента (FN) дороже, чем потратить ресурсы на удержание лояльного (FP).

**Метрика: Recall** — хотим найти максимум клиентов, которые уйдут.

## 6. Модель 1: Логистическая регрессия (Baseline)

In [ ]:
# Обучаем Logistic Regression
lr = LogisticRegression(
    class_weight='balanced',  # Учитываем дисбаланс
    max_iter=1000,
    random_state=42
)

lr.fit(X_train_scaled, y_train)
y_pred_lr = lr.predict(X_test_scaled)

print('=' * 50)
print('LOGISTIC REGRESSION')
print('=' * 50)
print('\nClassification Report:')
print(classification_report(y_test, y_pred_lr, target_names=['Existing', 'Attrited']))

In [ ]:
# Confusion Matrix
plt.figure(figsize=(6, 5))
cm_lr = confusion_matrix(y_test, y_pred_lr)
sns.heatmap(cm_lr, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Existing', 'Attrited'],
            yticklabels=['Existing', 'Attrited'])
plt.title('Confusion Matrix: Logistic Regression')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.tight_layout()
plt.show()

## 7. Модель 2: Random Forest

In [ ]:
# Обучаем Random Forest (не требует масштабирования)
rf = RandomForestClassifier(
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

print('=' * 50)
print('RANDOM FOREST (default params)')
print('=' * 50)
print('\nClassification Report:')
print(classification_report(y_test, y_pred_rf, target_names=['Existing', 'Attrited']))

In [ ]:
# Confusion Matrix
plt.figure(figsize=(6, 5))
cm_rf = confusion_matrix(y_test, y_pred_rf)
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Greens', 
            xticklabels=['Existing', 'Attrited'],
            yticklabels=['Existing', 'Attrited'])
plt.title('Confusion Matrix: Random Forest')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.tight_layout()
plt.show()

## 8. Подбор гиперпараметров (GridSearchCV)

In [ ]:
# Сетка параметров для Random Forest
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 15, None],
    'min_samples_split': [2, 5, 10]
}

print('Параметры для поиска:')
for param, values in param_grid.items():
    print(f'  {param}: {values}')

total_combinations = 1
for values in param_grid.values():
    total_combinations *= len(values)
print(f'\nВсего комбинаций: {total_combinations}')

In [ ]:
# Cross-validation strategy
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# GridSearchCV
grid_search = GridSearchCV(
    RandomForestClassifier(class_weight='balanced', random_state=42, n_jobs=-1),
    param_grid,
    cv=cv,
    scoring='recall',  # Оптимизируем по Recall
    n_jobs=-1,
    verbose=1
)

print('Запускаем GridSearchCV...')
grid_search.fit(X_train, y_train)
print('\nГотово!')

In [ ]:
print('=' * 50)
print('РЕЗУЛЬТАТЫ GRIDSEARCHCV')
print('=' * 50)
print(f'\nЛучшие параметры: {grid_search.best_params_}')
print(f'Лучший Recall на CV: {grid_search.best_score_:.4f}')

In [ ]:
# Финальная оценка лучшей модели
best_rf = grid_search.best_estimator_
y_pred_best = best_rf.predict(X_test)

print('=' * 50)
print('RANDOM FOREST (tuned)')
print('=' * 50)
print('\nClassification Report:')
print(classification_report(y_test, y_pred_best, target_names=['Existing', 'Attrited']))

In [ ]:
# Confusion Matrix для лучшей модели
plt.figure(figsize=(6, 5))
cm_best = confusion_matrix(y_test, y_pred_best)
sns.heatmap(cm_best, annot=True, fmt='d', cmap='Oranges', 
            xticklabels=['Existing', 'Attrited'],
            yticklabels=['Existing', 'Attrited'])
plt.title('Confusion Matrix: Random Forest (tuned)')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.tight_layout()
plt.show()

## 9. Сравнение моделей

In [ ]:
# Собираем метрики всех моделей
def get_metrics(y_true, y_pred):
    return {
        'Accuracy': accuracy_score(y_true, y_pred),
        'Recall': recall_score(y_true, y_pred),
        'Precision': precision_score(y_true, y_pred),
        'F1': f1_score(y_true, y_pred)
    }

results = pd.DataFrame([
    {'Модель': 'Logistic Regression', **get_metrics(y_test, y_pred_lr)},
    {'Модель': 'Random Forest', **get_metrics(y_test, y_pred_rf)},
    {'Модель': 'Random Forest (tuned)', **get_metrics(y_test, y_pred_best)}
])

# Форматируем для красивого вывода
results_display = results.copy()
for col in ['Accuracy', 'Recall', 'Precision', 'F1']:
    results_display[col] = results_display[col].apply(lambda x: f'{x:.4f}')

print('=' * 80)
print('СРАВНЕНИЕ МОДЕЛЕЙ')
print('=' * 80)
print(results_display.to_string(index=False))
print('\n* Основная метрика: Recall')

In [ ]:
# Визуализация сравнения
fig, axes = plt.subplots(1, 4, figsize=(16, 4))
metrics = ['Accuracy', 'Recall', 'Precision', 'F1']
colors = ['#3498db', '#2ecc71', '#e74c3c']

for i, metric in enumerate(metrics):
    ax = axes[i]
    bars = ax.bar(results['Модель'], results[metric], color=colors, edgecolor='black')
    ax.set_title(metric, fontsize=12)
    ax.set_ylim(0, 1)
    ax.tick_params(axis='x', rotation=45)
    
    # Подписи значений
    for bar, val in zip(bars, results[metric]):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
                f'{val:.3f}', ha='center', fontsize=10)

plt.suptitle('Сравнение метрик моделей', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

## 10. Важность признаков

In [ ]:
# Feature importances из лучшей модели Random Forest
importances = best_rf.feature_importances_

feat_imp = pd.DataFrame({
    'feature': X.columns,
    'importance': importances
}).sort_values('importance', ascending=False)

print('Top 10 важных признаков:')
print(feat_imp.head(10).to_string(index=False))

In [ ]:
# Визуализация важности признаков
plt.figure(figsize=(10, 8))
top_n = 15
top_features = feat_imp.head(top_n)

plt.barh(range(top_n), top_features['importance'], color='steelblue', edgecolor='black')
plt.yticks(range(top_n), top_features['feature'])
plt.xlabel('Важность')
plt.title(f'Top {top_n} важных признаков (Random Forest)', fontsize=14)
plt.gca().invert_yaxis()

# Добавляем значения
for i, v in enumerate(top_features['importance']):
    plt.text(v + 0.005, i, f'{v:.3f}', va='center', fontsize=9)

plt.tight_layout()
plt.show()

## 11. Выводы

In [ ]:
print('=' * 80)
print('ИТОГОВЫЕ ВЫВОДЫ')
print('=' * 80)

print('''
1. EDA ПОКАЗАЛ:
   - Датасет: 10,127 клиентов, 18 признаков (после удаления служебных)
   - Дисбаланс классов: ~16% ушедших vs ~84% оставшихся
   - Сильные корреляции между транзакционными признаками
   - Пропусков нет

2. ВЫБРАНА МЕТРИКА RECALL:
   - Важнее найти всех потенциально уходящих клиентов
   - Пропустить уходящего клиента (FN) дороже, чем лишнее удержание (FP)

3. СРАВНИЛИ 2 МОДЕЛИ:
   - Logistic Regression (baseline)
   - Random Forest (с подбором гиперпараметров через GridSearchCV)
''')

# Лучшая модель
best_model_idx = results['Recall'].idxmax()
best_model_name = results.loc[best_model_idx, 'Модель']
best_recall = results.loc[best_model_idx, 'Recall']
best_f1 = results.loc[best_model_idx, 'F1']
best_accuracy = results.loc[best_model_idx, 'Accuracy']

print(f'''4. ЛУЧШАЯ МОДЕЛЬ: {best_model_name}
   - Параметры: {grid_search.best_params_}
   - Recall: {best_recall:.4f}
   - F1-score: {best_f1:.4f}
   - Accuracy: {best_accuracy:.4f}

5. КЛЮЧЕВЫЕ ПРИЗНАКИ (по важности):
   - {feat_imp.iloc[0]['feature']}: {feat_imp.iloc[0]['importance']:.3f}
   - {feat_imp.iloc[1]['feature']}: {feat_imp.iloc[1]['importance']:.3f}
   - {feat_imp.iloc[2]['feature']}: {feat_imp.iloc[2]['importance']:.3f}

6. РЕКОМЕНДАЦИИ:
   - Обратить внимание на клиентов с низким Total_Trans_Ct (кол-во транзакций)
   - Мониторить Total_Revolving_Bal и Total_Trans_Amt
   - Модель готова к использованию для раннего выявления оттока
''')

---

**Ссылка на датасет**: https://www.kaggle.com/datasets/whenamancodes/credit-card-customers-prediction